# Preparing a state with antiferromagnetic order in the Ising model

This notebook illustrates how to use Pulser to build a sequence for studying an antiferromagnetic state in an Ising-like model. It is based on [10.1103/PhysRevX.8.021070](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.021070), where arrays of Rydberg atoms were programmed and whose correlations were studied.

We begin by importing some basic modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip

from pulser import Pulse, Sequence, Register
from pulser_simulation import QutipEmulator
from pulser.waveforms import RampWaveform

from pulser_myqlm import FresnelQPU, IsingAQPU

Working with the Pulser `Device` associated with the QPU:

In [ ]:
FRESNEL_QPU = FresnelQPU(None)  # mimics QPU with pulser-simulation as backend
FRESNEL_DEVICE = FRESNEL_QPU.device  # the Device implemented by the QPU

Define if simulations should be performed locally (by setting LOCAL_SIMULATIONS to True) or on a remote emulator (AnalogQPU, by setting LOCAL_SIMULATIONS to False):

In [ ]:
# If LOCAL_SIMULATIONS is True: Uses IsingAPQU (pulser-simulation) locally
# Otherwise: Uses AnalogQPU (if it can be imported)
LOCAL_SIMULATIONS = True


# Checking AnalogQPU can be imported
if not LOCAL_SIMULATIONS:
    try:
        from qlmaas.qpus import AnalogQPU
    except ImportError as e:
        raise ImportError(
            "Can't import AnalogQPU: simulations can only be performed locally using IsingAQPU (uses pulser-simulation)."
        ) from e

Simulation parameters define number of shots and whether or not modulation of the pulses should be taken into account:

In [ ]:
NBSHOTS = 0  # must be 0 for AnalogQPU
# If 0 and local simulations, default internally to 2000
MODULATION = False  # Whether or not to use Modulated Sequence in the simulation

if not LOCAL_SIMULATIONS and NBSHOTS > 0:
    raise ValueError("Simulation with AnalogQPU: number of shots must be 0.")

## Waveforms 

We are realizing the following program

<center>
<img src="attachment:AF_Ising_program.png" alt="AF Pulse Sequence" width="300">
</center>

The pulse and the register are defined by the following parameters:

In [ ]:
# Parameters in rad/µs and ns
Omega_max = 12.56637061435917  # 2*2*pi
U = Omega_max / 2.0

delta_0 = -6 * U
delta_f = 2 * U

t_rise = 252
t_fall = 500
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000

R_interatomic = FRESNEL_DEVICE.rydberg_blockade_radius(U)

N_side = 3
reg = Register.square(N_side, R_interatomic, prefix="q")
print(f"Interatomic Radius is: {R_interatomic}µm.")
reg.draw()

## Creating my sequence

We compose our pulse with the following objects from Pulser:

In [ ]:
rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0.0, Omega_max), delta_0, 0.0)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.0)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.0), delta_f, 0.0)

In [ ]:
seq = Sequence(reg, FRESNEL_DEVICE)
seq.declare_channel("ising", "rydberg_global")

seq.add(rise, "ising")
seq.add(sweep, "ising")
seq.add(fall, "ising")

seq.draw()

## Phase Diagram

The pulse sequence travels though the following path in the phase diagram of the system (the shaded area represents the antiferromagnetic phase):

In [ ]:
delta = []
omega = []
for x in seq._schedule["ising"]:
    if isinstance(x.type, Pulse):
        omega += list(x.type.amplitude.samples / U)
        delta += list(x.type.detuning.samples / U)

fig, ax = plt.subplots()
ax.grid(True, which="both")

ax.set_ylabel(r"$\hbar\delta(t)/U$", fontsize=16)
ax.set_xlabel(r"$\hbar\Omega(t)/U$", fontsize=16)
ax.set_xlim(0, 3)
ax.axhline(y=0, color="k")
ax.axvline(x=0, color="k")

y = np.arange(0.0, 6, 0.01)
x = 1.522 * (1 - 0.25 * (y - 2) ** 2)
ax.fill_between(x, y, alpha=0.4)

ax.plot(omega, delta, "red", lw=2)
plt.show()

## Simulation: Spin-Spin Correlation Function

We shall now evaluate the quality of the obtained state by calculating the *spin-spin correlation function*, defined as:


$$g^c(k,l)= \frac{1}{N_{k,l}}\sum_{(i,j) = (kR,lR)} \left[ \langle n_i n_j \rangle - \langle n_i \rangle \langle n_j \rangle \right]$$

where the $c$ indicates that we are calculating the *connected* part, and where the sum is over all pairs $(i,j)$ whose distance is ${\bf r}_i - {\bf r}_j = (k R,l R)$ in the atomic array coordinate (both $k$ and $l$ are positive or negative integers within the size of the array).

We run a simulation of the sequence:

In [ ]:
job = IsingAQPU.convert_sequence_to_job(seq, nbshots=NBSHOTS, modulation=MODULATION)

In [ ]:
MyQLMPulserSimBackend = IsingAQPU.from_sequence(seq, qpu=None)
MYQLM_BACKEND = MyQLMPulserSimBackend if LOCAL_SIMULATIONS else AnalogQPU()

In [ ]:
results = MYQLM_BACKEND.submit(job)

Sample from final state using `sample_final_state()` method:

In [ ]:
def get_samples_from_result(result):
    """Converting the MyQLM Results into Pulser Samples"""
    samples = {}
    n_qubits = N_side**2
    for sample in result.raw_data:
        if len(sample.state.bitstring) > n_qubits:
            raise ValueError(
                f"State {sample.state} is incompatible with number of qubits"
                f" declared {n_qubits}."
            )
        counts = sample.probability
        samples[sample.state.bitstring.zfill(n_qubits)] = counts
    return samples

In [ ]:
count = get_samples_from_result(results)

most_freq = {k: v for k, v in count.items() if v > 10 / 1000}
plt.bar(list(most_freq.keys()), list(most_freq.values()))
plt.xticks(rotation="vertical")
plt.show()